### For creating points from Latitude and Longitude

In [ ]:
locations = df[['lat', 'lon']]
locationlist = locations.values.tolist()
len(locationlist)
locationlist[7]
# Output: [35.69798843112353, 139.6550454876133]

### For Creating Trajectories with Folium

In [ ]:
import folium

m = folium.Map(location=[34.95, 137.16],
              zoom_start=15)

loc = locationlist

folium.PolyLine(loc,
                color='red',
                weight=1,
                opacity=0.8).add_to(m)
m